In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os

# os.chdir('D:/Desktop/Flex 4/7074 Data Mining 2/Final Project/ml-latest-small/')
os.chdir('C:/UCin/Sem 2/Data Mining II/Project/ml-latest-small')

In [3]:
os.getcwd

<function nt.getcwd()>

In [4]:
pd.read_csv('movies.csv')

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [5]:
# Reading ratings file
# Ignore the timestamp column
ratings = pd.read_csv('ratings.csv', usecols=['rating','userId', 'movieId'])

# Reading users file
#users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading movies file
movies = pd.read_csv('movies.csv', usecols=['movieId', 'title', 'genres'])

In [6]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
userId     100836 non-null int64
movieId    100836 non-null int64
rating     100836 non-null float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [8]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
movieId    9742 non-null int64
title      9742 non-null object
genres     9742 non-null object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [10]:
# Break up the big genre string into a string array
movies['genres'] = movies['genres'].str.split('|')
# Convert genres to string value
movies['genres'] = movies['genres'].fillna("").astype('str')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
1,2,Jumanji (1995),"['Adventure', 'Children', 'Fantasy']"
2,3,Grumpier Old Men (1995),"['Comedy', 'Romance']"
3,4,Waiting to Exhale (1995),"['Comedy', 'Drama', 'Romance']"
4,5,Father of the Bride Part II (1995),['Comedy']


In [11]:
movies['release_year'] = movies['title'].str[-6:]
movies.head()

,movieId,title,genres,release_year
0,1,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",(1995)
1,2,Jumanji (1995),"['Adventure', 'Children', 'Fantasy']",(1995)
2,3,Grumpier Old Men (1995),"['Comedy', 'Romance']",(1995)
3,4,Waiting to Exhale (1995),"['Comedy', 'Drama', 'Romance']",(1995)
4,5,Father of the Bride Part II (1995),['Comedy'],(1995)


In [12]:
#movies.apply(lambda x: [x['genres']] + list(x['release_year']), 1)
#movies.genres.append(movies['release_year'])
movies['genres2'] = movies['genres'] + list(movies['release_year'])
movies.head()

,movieId,title,genres,release_year,genres2
0,1,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",(1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
1,2,Jumanji (1995),"['Adventure', 'Children', 'Fantasy']",(1995),"['Adventure', 'Children', 'Fantasy'](1995)"
2,3,Grumpier Old Men (1995),"['Comedy', 'Romance']",(1995),"['Comedy', 'Romance'](1995)"
3,4,Waiting to Exhale (1995),"['Comedy', 'Drama', 'Romance']",(1995),"['Comedy', 'Drama', 'Romance'](1995)"
4,5,Father of the Bride Part II (1995),['Comedy'],(1995),['Comedy'](1995)


In [13]:
movies.genres

0       ['Adventure', 'Animation', 'Children', 'Comedy...
1                    ['Adventure', 'Children', 'Fantasy']
2                                   ['Comedy', 'Romance']
3                          ['Comedy', 'Drama', 'Romance']
4                                              ['Comedy']
                              ...                        
9737         ['Action', 'Animation', 'Comedy', 'Fantasy']
9738                   ['Animation', 'Comedy', 'Fantasy']
9739                                            ['Drama']
9740                              ['Action', 'Animation']
9741                                           ['Comedy']
Name: genres, Length: 9742, dtype: object

### CONTENT BASED FILTERING
-We have used the genre of a movie to recommend similar movies to a user <br>
-Used Term frequency (TF) and Inverse Document Frequency (IDF) to identify the genres and release year of a given movie and assign a cosine similarity score to get Top 20 recommendations

Code reference - https://github.com/khanhnamle1994/movielens/blob/master/Content_Based_and_Collaborative_Filtering_Models.ipynb

__Use TfidfVectorizer function from scikit-learn, which transforms text to feature vectors that can be used as input to estimator.__

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres2'])
tfidf_matrix.shape

(9742, 1478)

__Ue the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, we will use sklearn's linear_kernel instead of cosine_similarities since it is much faster.__

In [15]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.60701112, 0.17872226, 0.16965875],
       [0.60701112, 1.        , 0.16888819, 0.16032339],
       [0.17872226, 0.16888819, 1.        , 0.76986467],
       [0.16965875, 0.16032339, 0.76986467, 1.        ]])

__Write a function that returns the 20 most similar movies based on the cosine similarity score.__

In [16]:
# Build a 1-dimensional array with movie titles
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

__Test the recommendation for a movie__

In [17]:
movies[movies.title=='Avengers: Infinity War - Part I (2018)']

,movieId,title,genres,release_year,genres2
8693,122912,Avengers: Infinity War - Part I (2018),"['Action', 'Adventure', 'Sci-Fi']",(2018),"['Action', 'Adventure', 'Sci-Fi'](2018)"


In [18]:
# the recommendations work well as they recommend movies with similar genres of the input movie based on the cosine similarity
genre_recommendations('Avengers: Infinity War - Part I (2018)').head(20)

9706                               Rampage (2018)
9710               Solo: A Star Wars Story (2018)
9713                  Ant-Man and the Wasp (2018)
9709                            Deadpool 2 (2018)
9712                   Sorry to Bother You (2018)
9705                 Pacific Rim: Uprising (2018)
9697                     A Wrinkle in Time (2018)
8489                     Jupiter Ascending (2015)
8686               Avengers: Age of Ultron (2015)
8689                               Ant-Man (2015)
8812                         Power/Rangers (2015)
9038                             Turbo Kid (2015)
9073                        Hardcore Henry (2015)
6743                              Iron Man (2008)
6791    Journey to the Center of the Earth (2008)
6825                     Mutant Chronicles (2008)
6963                             Outlander (2008)
8697                        Doctor Strange (2016)
8916          Independence Day: Resurgence (2016)
8917                      Star Trek Beyond (2016)


In [19]:
pd.set_option('display.max_colwidth', -1)
genre_recommendations('Toy Story (1995)').head(20)

209     Gordy (1995)                                           
160     Reckless (1995)                                        
4604    Ninja Scroll (Jûbei ninpûchô) (1995)                   
6948    Tale of Despereaux, The (2008)                         
6194    Wild, The (2006)                                       
7760    Asterix and the Vikings (Astérix et les Vikings) (2006)
3568    Monsters, Inc. (2001)                                  
8927    The Good Dinosaur (2015)                               
2355    Toy Story 2 (1999)                                     
6486    Shrek the Third (2007)                                 
9430    Moana (2016)                                           
2809    Adventures of Rocky and Bullwinkle, The (2000)         
3000    Emperor's New Groove, The (2000)                       
8219    Turbo (2013)                                           
1706    Antz (1998)                                            
1       Jumanji (1995)                  

In [20]:
genre_recommendations('Insidious: Chapter 3 (2015)').head(20)

8981    The Gallows (2015)                   
9206    Frankenstein (2015)                  
8867    Maggie (2015)                        
9360    Body (2015)                          
8989    Massu Engira Maasilamani (2015)      
9093    Into the Grizzly Maze (2015)         
8880    Return to Sender (2015)              
8955    Careful What You Wish For (2015)     
9071    Spotlight (2015)                     
9168    Mojave (2015)                        
9059    Knock Knock (2015)                   
8734    Zipper (2015)                        
8747    The Stanford Prison Experiment (2015)
8858    Partisan (2015)                      
9090    Bridge of Spies (2015)               
9108    The Perfect Guy (2015)               
6195    Silent Hill (2006)                   
7329    Nightmare on Elm Street, A (2010)    
7569    Insidious (2010)                     
4961    Paperhouse (1988)                    
Name: title, dtype: object

In [21]:
genre_recommendations('Welcome to Me (2014)').head(20)

8372    Grand Budapest Hotel, The (2014)                       
8407    Calvary (2014)                                         
8441    Million Dollar Arm (2014)                              
8452    Birdman: Or (The Unexpected Virtue of Ignorance) (2014)
8468    Angriest Man in Brooklyn, The (2014)                   
8485    The Hundred-Foot Journey (2014)                        
8507    Pride (2014)                                           
8521    This Is Where I Leave You (2014)                       
8571    Rudderless (2014)                                      
8579    The Longest Week (2014)                                
8634    Men, Women & Children (2014)                           
8748    A Pigeon Sat on a Branch Reflecting on Existence (2014)
8801    While We're Young (2014)                               
8833    The Road Within (2014)                                 
8852    Welcome to Me (2014)                                   
8861    Just Before I Go (2014)         

In [33]:
#movies[movies['title'].str.contains("2014")].tail(40)

### COLLABORATIVE FILTERING - USER-USER BASED

__Data processing__

In [34]:
# Fill NaN values in user_id and movie_id column with 0
ratings['userId'] = ratings['userId'].fillna(0)
ratings['movieId'] = ratings['movieId'].fillna(0)

# Replace NaN values in rating column with average of all values
ratings['rating'] = ratings['rating'].fillna(ratings['rating'].mean())

__Take random sample of data__

In [35]:
# Randomly sample 20% of the ratings dataset
small_data = ratings.sample(frac=0.2,random_state=100)
# Check the sample info
print(small_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20167 entries, 13300 to 90335
Data columns (total 3 columns):
userId     20167 non-null int64
movieId    20167 non-null int64
rating     20167 non-null float64
dtypes: float64(1), int64(2)
memory usage: 630.2 KB
None


In [36]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(small_data, test_size=0.2)

In [37]:
# Create two user-item matrices, one for training and another for testing
train_data_matrix = train_data.to_numpy()
test_data_matrix = test_data.to_numpy()

# Check their shape
print(train_data_matrix.shape)
print(test_data_matrix.shape)

(16133, 3)
(4034, 3)


In [38]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation[:4, :4])

[[1.         0.99053286 0.08773421 0.99812431]
 [0.99053286 1.         0.2236501  0.99707892]
 [0.08773421 0.2236501  1.         0.14855331]
 [0.99812431 0.99707892 0.14855331 1.        ]]


In [39]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation[:4, :4])

[[ 1.          0.00625342 -0.05891796]
 [ 0.00625342  1.          0.00455255]
 [-0.05891796  0.00455255  1.        ]]


In [40]:
# Function to predict ratings
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [41]:

from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [42]:
# Predict ratings on the training data with both similarity score
user_prediction = predict(train_data_matrix, user_correlation, type='user')
item_prediction = predict(train_data_matrix, item_correlation, type='item')

# RMSE on the test data
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 24893.65930295704
Item-based CF RMSE: 29664.060421716946


In [43]:
predict(train_data_matrix, user_correlation, type='user')
train_data_matrix - train_data_matrix.mean(axis=1)[:, np.newaxis]

array([[-1.52996667e+04,  3.08163333e+04, -1.55166667e+04],
       [-2.71000000e+02,  7.20000000e+02, -4.49000000e+02],
       [ 2.61666667e+02,  2.66666667e+01, -2.88333333e+02],
       ...,
       [-2.83333333e+02,  1.13966667e+03, -8.56333333e+02],
       [ 1.21333333e+02,  1.41333333e+02, -2.62666667e+02],
       [-3.42130000e+04,  6.87760000e+04, -3.45630000e+04]])

### Singular Value Decomposition

In [44]:
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')

In [45]:
n_users = ratings.userId.unique().shape[0]
n_movies = ratings.movieId.unique().shape[0]
print ('Number of users =' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users =610 | Number of movies = 9724


In [46]:
Ratings = ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
Ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
R = Ratings.to_numpy()
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [48]:
sparsity = round(1.0 - len(ratings) / float(n_users * n_movies), 3)
print ('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%')

The sparsity level of MovieLens1M dataset is 98.3%


In [49]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

In [50]:
sigma = np.diag(sigma)

In [51]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [52]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,2.167328,0.402751,0.840184,-0.076281,-0.551337,2.504091,-0.890114,-0.026443,0.196974,1.593259,...,-0.023453,-0.019967,-0.026939,-0.026939,-0.023453,-0.026939,-0.023453,-0.023453,-0.023453,-0.058732
1,0.211459,0.006658,0.033455,0.017419,0.183430,-0.062473,0.083037,0.024158,0.049330,-0.152530,...,0.019498,0.016777,0.022219,0.022219,0.019498,0.022219,0.019498,0.019498,0.019498,0.032281
2,0.003588,0.030518,0.046393,0.008176,-0.006247,0.107328,-0.012416,0.003779,0.007297,-0.059362,...,0.005909,0.006209,0.005610,0.005610,0.005909,0.005610,0.005909,0.005909,0.005909,0.008004
3,2.051549,-0.387104,-0.252199,0.087562,0.130465,0.270210,0.477835,0.040313,0.025858,-0.017365,...,0.004836,0.004172,0.005500,0.005500,0.004836,0.005500,0.004836,0.004836,0.004836,-0.023311
4,1.344738,0.778511,0.065749,0.111744,0.273144,0.584426,0.254930,0.128788,-0.085541,1.023455,...,-0.008042,-0.007419,-0.008664,-0.008664,-0.008042,-0.008664,-0.008042,-0.008042,-0.008042,-0.010127


In [53]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.userId == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print ('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [54]:
already_rated, predictions = recommend_movies(preds, 400, movies, ratings, 20)
#Change 400 from 1 to 610 = # unique userId

User 400 has already rated 43 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [55]:
already_rated.head(20)

,userId,movieId,rating,timestamp,title,genres
0,400,6,5.0,1498870480,Heat (1995),Action|Crime|Thriller
10,400,608,5.0,1498870431,Fargo (1996),Comedy|Crime|Drama|Thriller
1,400,47,5.0,1498870391,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
26,400,3949,5.0,1498870465,Requiem for a Dream (2000),Drama
18,400,1210,5.0,1498870163,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
30,400,44199,5.0,1498870195,Inside Man (2006),Crime|Drama|Thriller
15,400,1196,5.0,1498870150,Star Wars: Episode V - The Empire Strikes Back (1980),Action|Adventure|Sci-Fi
38,400,79132,5.0,1498870346,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX
12,400,858,5.0,1498870339,"Godfather, The (1972)",Crime|Drama
23,400,2571,5.0,1498870160,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [56]:
# Top 20 movies that User 400 hopefully will enjoy
predictions

,movieId,title,genres
453,527,Schindler's List (1993),Drama|War
4108,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
2121,2858,American Beauty (1999),Drama|Romance
1711,2329,American History X (1998),Crime|Drama
1480,2028,Saving Private Ryan (1998),Action|Drama|War
814,1089,Reservoir Dogs (1992),Crime|Mystery|Thriller
94,110,Braveheart (1995),Action|Drama|War
967,1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
3114,4226,Memento (2000),Mystery|Thriller
7729,91529,"Dark Knight Rises, The (2012)",Action|Adventure|Crime|IMAX


In [58]:
#!pip install surprise

In [98]:
# Import libraries from Surprise package
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(small_data[['userId', 'movieId', 'rating']], reader)

# Split the dataset for 5-fold evaluation
#data = data.build_full_trainset()
#data.split(n_folds=5)

In [99]:
# Use the SVD algorithm.
svd = SVD()

# Compute the RMSE of the SVD algorithm.
cross_validate(svd, data, measures=['RMSE'])

{'test_rmse': array([0.90807901, 0.93732377, 0.91208012, 0.94544492, 0.92118803]),
 'fit_time': (2.058058500289917,
  2.1552720069885254,
  3.2479970455169678,
  2.2957229614257812,
  2.807504177093506),
 'test_time': (0.047595977783203125,
  0.07030057907104492,
  0.1133420467376709,
  0.058730363845825195,
  0.019946575164794922)}

In [97]:
#trainset = data.build_full_trainset()
#svd.train(trainset)

In [96]:
ratings[ratings['userId'] == 400].head()

,userId,movieId,rating,timestamp
61156,400,6,5.0,1498870480
61157,400,47,5.0,1498870391
61158,400,50,5.0,1498870285
61159,400,260,5.0,1498870148
61160,400,293,5.0,1498870424


In [90]:
svd.predict(400, 1994)

Prediction(uid=400, iid=1994, r_ui=None, est=4.004011476699977, details={'was_impossible': False})

In [93]:
#ratings.movieId.unique()

In [94]:
#ratings.userId.unique()